In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import constants as ct

In [22]:
df_leitos = pd.read_csv(ct.PATH_LEITOS)

In [23]:
df_leitos

,CO_UNIDADE,CO_TIPO_LEITO,QT_EXIST,QT_SUS,DS_LEITO
0,5211502589265,4,2,0,OBSTETRICIA CLINICA
1,2100472462192,1,5,5,CIRURGIA GERAL
2,2100472462192,2,18,18,CLINICA GERAL
3,2100472462192,5,6,6,PEDIATRIA CLINICA
4,2106702461838,1,4,4,CIRURGIA GERAL
...,...,...,...,...,...
55710,4313402232057,3,12,0,UTI ADULTO - TIPO II
55711,4317203017060,2,1,1,NEFROUROLOGIA
55712,2408102409194,2,6,6,HEMATOLOGIA
55713,2910802802031,7,3,3,CIRURGICO/DIAGNOSTICO/TERAPEUTICO


In [3]:
df_gripe = pd.read_csv(ct.PATH_GRIPE, low_memory=False)

Unique por coluna

In [26]:
{x:len(df_gripe[x].unique()) for x in df_gripe.columns}

{'Número da notificação': 1,
 'Data da notificação': 92,
 'Semana epidemiológica da notificação': 120,
 'Data dos primeiros sintomas': 20,
 'Semana epidemiológica dos primeiros sintomas': 131,
 'UF notificante': 20,
 'Município notificante': 17,
 'Código do município notificante': 107,
 'Unidade notificante': 107,
 'CNES da unidade notificante': 434,
 'Sexo': 447,
 'Data de nascimento': 3,
 'Unidade da idade': 12594,
 'Idade': 3,
 'Gestante': 107,
 'Raça/cor': 8,
 'Etnia': 7,
 'Escolaridade': 2,
 'Ocupação': 8,
 'CBO da ocupação': 118,
 'UF de residência': 116,
 'Município de residência': 18,
 'Código do município de residência': 110,
 'Bairro': 114,
 'Zona': 1886,
 'País de residência': 9,
 'Código do país de residência': 9,
 'Histórico de viagem internacional': 8,
 'País viagem internacional': 6,
 'Código país viagem internacional': 18,
 'Local viagem internacional': 17,
 'Data de partida viagem internacional': 25,
 'Data de retorno viagem internacional': 38,
 'Caso de surto de SG qu

In [4]:
df_gripe.shape

(17545, 162)

In [11]:
df_gripe.isna().sum(axis=0)

Número da notificação                               0
Data da notificação                                 0
Semana epidemiológica da notificação                0
Data dos primeiros sintomas                         0
Semana epidemiológica dos primeiros sintomas        0
                                                ...  
Unnamed: 157                                    17542
Unnamed: 158                                    17544
Unnamed: 159                                    17544
Unnamed: 160                                    17544
Unnamed: 161                                    17544
Length: 162, dtype: int64

In [24]:
df_gripe['Unidade notificante'].unique()

array([330455, 330580, 330330, 330452, 330170, 330020, 330550, 330070,
       330200, 330620, 330350, 330285, 330520, 330490, 330610, 330390,
       330240, 221100, 330510, 330050, 330010, 330022, 330030, 330340,
       330430, 330411, 330630, 330100, 330470, 330190, 330575, 330110,
       355030, 330420, 530010, 330093, 330400, 330187, 330475, 330415,
       330220, 330210, 330515, 330130, 330600, 330080, 330270, 330320,
       330500, 330290, 330250, 330360, 330045, 330040, 330395, 314390,
       316370, 330540, 330185, 330060, 311340, 330090, 330280, 310150,
       355410, 330023, 330180, 330205, 330380, 292740, 150270, 330015,
       350390, 230440, 520620, 291920, 330370, 320120, 270430, 310620,
       316250, 330460, 330440, 354990, 313240, 510340, 420820, 312510,
       316720, 313670, 211130, 431490, 330225, 354850, 351340, 330150,
       410840, 351880, 355220, 354980, 350950, 351380, 420200, 320530,
       410420, 330310, 280030])

In [30]:
df_gripe['Semana epidemiológica da notificação'] = pd.to_datetime(df_gripe['Semana epidemiológica da notificação'])

In [31]:
df_gripe['Semana epidemiológica da notificação'].min()

Timestamp('2020-01-01 00:00:00')

In [32]:
df_gripe['Semana epidemiológica da notificação'].max()

Timestamp('2020-12-05 00:00:00')

In [5]:
df_gripe.sample(3)

,Número da notificação,Data da notificação,Semana epidemiológica da notificação,Data dos primeiros sintomas,Semana epidemiológica dos primeiros sintomas,UF notificante,Município notificante,Código do município notificante,Unidade notificante,CNES da unidade notificante,...,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161
13817,3,1586900000000000,12/04/2020,16,07/04/2020,15,RJ,CABO FRIO,330070,HOSPITAL SANTA IZABEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9907,3,1587800000000000,14/04/2020,16,11/04/2020,15,RJ,SAO GONCALO,330490,HOSPITAL SAO GONCALO LTDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14816,3,1588900000000000,28/04/2020,18,27/04/2020,18,RJ,RIO DE JANEIRO,330455,UNIMED RIO PA BARRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_gripe['Número da notificação'].unique()

array([3])

In [12]:
list(df_gripe.columns)

['Número da notificação',
 'Data da notificação',
 'Semana epidemiológica da notificação',
 'Data dos primeiros sintomas',
 'Semana epidemiológica dos primeiros sintomas',
 'UF notificante',
 'Município notificante',
 'Código do município notificante',
 'Unidade notificante',
 'CNES da unidade notificante',
 'Sexo',
 'Data de nascimento',
 'Unidade da idade',
 'Idade',
 'Gestante',
 'Raça/cor',
 'Etnia',
 'Escolaridade',
 'Ocupação',
 'CBO da ocupação',
 'UF de residência',
 'Município de residência',
 'Código do município de residência',
 'Bairro',
 'Zona',
 'País de residência',
 'Código do país de residência',
 'Histórico de viagem internacional',
 'País viagem internacional',
 'Código país viagem internacional',
 'Local viagem internacional',
 'Data de partida viagem internacional',
 'Data de retorno viagem internacional',
 'Caso de surto de SG que evoluiu para SRAG',
 'Caso nosocomial - adquirido no hospital',
 'Contato com aves ou suínos',
 'Sintomas - febre',
 'Sintomas - tosse'

In [10]:
df_gripe.isna().sum(axis=0)

Número da notificação                               0
Data da notificação                                 0
Semana epidemiológica da notificação                0
Data dos primeiros sintomas                         0
Semana epidemiológica dos primeiros sintomas        0
                                                ...  
Unnamed: 157                                    17542
Unnamed: 158                                    17544
Unnamed: 159                                    17544
Unnamed: 160                                    17544
Unnamed: 161                                    17544
Length: 162, dtype: int64